In [3]:
import pyspark
from pyspark.sql import SparkSession

# local[*] creates a local spark cluster
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/05 21:57:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read \
    .option("header","true") \
    .option("recursiveFileLookup","true") \
    .parquet('data/pq/green')

In [6]:
df_green.createOrReplaceTempView('green')

In [10]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) as hour,
    PULocationId as zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [11]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|            295.34|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [12]:
df_green_revenue.write \
    .parquet('data/report/hourly_revenue/green')

25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/05 22:22:15 WARN MemoryManager: Total allocation exceeds 95.0

In [19]:
!ls -lahR data/report/hourly_revenue

total 0
drwxr-xr-x@  3 romadv-mac  staff    96B  5 Jul 22:22 .
drwxr-xr-x@  4 romadv-mac  staff   128B  5 Jul 22:22 ..
drwxr-xr-x@ 26 romadv-mac  staff   832B  5 Jul 22:22 green

data/report/hourly_revenue/green:
total 7072
-rw-r--r--@  1 romadv-mac  staff     0B  5 Jul 22:22 _SUCCESS
drwxr-xr-x@ 26 romadv-mac  staff   832B  5 Jul 22:22 .
-rw-r--r--@  1 romadv-mac  staff     8B  5 Jul 22:22 ._SUCCESS.crc
drwxr-xr-x@  3 romadv-mac  staff    96B  5 Jul 22:22 ..
-rw-r--r--@  1 romadv-mac  staff   2.7K  5 Jul 22:22 .part-00000-2b0bae84-1431-4698-8dfc-5f18a37dab5a-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff   2.6K  5 Jul 22:22 .part-00001-2b0bae84-1431-4698-8dfc-5f18a37dab5a-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff   2.6K  5 Jul 22:22 .part-00002-2b0bae84-1431-4698-8dfc-5f18a37dab5a-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff   2.2K  5 Jul 22:22 .part-00003-2b0bae84-1431-4698-8dfc-5f18a37dab5a-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff

In [20]:
df_yellow = spark.read \
    .option("header","true") \
    .option("recursiveFileLookup","true") \
    .parquet('data/pq/yellow')

df_yellow.createOrReplaceTempView('yellow')

df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) as hour,
    PULocationId as zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

df_yellow_revenue.show()

[Stage 16:==============>                                         (4 + 11) / 15]

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000003|            57|
|2020-01-01 00:00:00|   7| 455.1700000000001|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.7999999999997|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24| 754.9500000000003|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [26]:
df_yellow_revenue.repartition(10) \
    .write \
    .parquet('data/report/hourly_revenue/yellow', mode='overwrite')

25/07/05 22:30:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/05 22:30:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/05 22:30:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/05 22:30:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/05 22:30:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [27]:
!ls -lahR data/report/hourly_revenue/yellow

total 26504
-rw-r--r--@  1 romadv-mac  staff     0B  5 Jul 22:30 _SUCCESS
drwxr-xr-x@ 24 romadv-mac  staff   768B  5 Jul 22:30 .
-rw-r--r--@  1 romadv-mac  staff     8B  5 Jul 22:30 ._SUCCESS.crc
drwxr-xr-x@  4 romadv-mac  staff   128B  5 Jul 22:30 ..
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00000-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00001-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00002-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00003-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00004-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parquet.crc
-rw-r--r--@  1 romadv-mac  staff    10K  5 Jul 22:30 .part-00005-2993c9a0-1474-49a8-8cc3-d5383f027653-c000.snappy.parq

In [29]:
# join green and yellow tables
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [30]:
df_join.show(10)

[Stage 48:================================================>       (13 + 2) / 15]

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  17|      195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  18|         7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  36|      295.34|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  48|        NULL|                NULL|10773.360000000026|                  455|
|2020-01-01 00:00:00|  51|        17.8|                   2|              31.0|                    1|
|2020-01-01 00:00:00| 130|      133.35|                   7|              NULL|                 NULL|
|2020-01-01 00:00:00| 143|       55.95|                   2|2142.0599999999986|   

In [31]:
df_join.write \
    .parquet('data/report/hourly_revenue/total')

25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/07/05 22:49:05 WARN MemoryManager: Total allocation exceeds 95.

In [38]:
# join with zones dimension table
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/lookup/taxi_zone_lookup.csv')

In [39]:
df_zones.show()

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [44]:
df_result = df_join.join(df_zones, 
             df_join.zone==df_zones.locationid)
df_result = df_result.drop('locationid')

In [46]:
df_result.show(5)

[Stage 73:==================>                                     (5 + 10) / 15]

+-------------------+----+------------+--------------------+------------------+---------------------+---------+--------------+------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|  borough|          zone|service_zone|
+-------------------+----+------------+--------------------+------------------+---------------------+---------+--------------+------------+
|2020-01-01 00:00:00|  17|      195.03|                   9|220.20999999999998|                    8| Brooklyn|       Bedford|   Boro Zone|
|2020-01-01 00:00:00|  18|         7.8|                   1|               5.8|                    1|    Bronx|  Bedford Park|   Boro Zone|
|2020-01-01 00:00:00|  36|      295.34|                  11|            109.17|                    3| Brooklyn|Bushwick North|   Boro Zone|
|2020-01-01 00:00:00|  48|        NULL|                NULL|10773.360000000026|                  455|Manhattan|  Clinton East| Yellow Zone|
|2020-01-01 00:00:00

In [47]:
df_result

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint, borough: string, zone: string, service_zone: string]

In [48]:
spark.stop()